In [ ]:
import numpy as np
import re
from functools import partial
import copy

In [ ]:
test_text = """Register A: 729
Register B: 0
Register C: 0

Program: 0,1,5,4,3,0"""

In [ ]:
with open("input.txt") as f:
    input_text = f.read()

In [ ]:


class Computer:
    def __init__(self, A, B, C, instructions):
        self.A = A
        self.B = B
        self.C = C
        self.instructions = instructions
        self.ptr = 0
        self.outputs = []
    
    def ptr_inc(func):
        def wrapper(self, *args, **kwargs):
            result = func(self, *args, **kwargs)
            self.ptr += 2
            return result
        return wrapper
    
    def combo(self, operand):
        match operand:
            case 4:
                return self.A
            case 5:
                return self.B
            case 6:
                return self.C
        return operand
    
    def _opcode(self, opcode):
        match opcode:
            case 0:
                return self.dva
            case 1:
                return self.bxl
            case 2:
                return self.bst
            case 3:
                return self.jnz
            case 4:
                return self.bxc
            case 5:
                return self.out
            case 6:
                return self.dvb
            case 7:
                return self.dvc
            case _:
                return
    
    @ptr_inc
    def dva(self, operand):
        self.A = int(self.A / (2 ** self.combo(operand)))

    @ptr_inc
    def dvb(self, operand):
        self.B = int(self.A / (2 ** self.combo(operand)))
        
    @ptr_inc
    def dvc(self, operand):
        self.C = int(self.A / (2 ** self.combo(operand)))

    @ptr_inc
    def bxl(self, operand):
        self.B = int(operand ^ self.B)
    
    @ptr_inc
    def bst(self, operand):
        self.B = int(self.combo(operand) % 8)
    
    def jnz(self, operand):
        if self.A == 0:
            self.ptr += 2
            return
        self.ptr = operand
    
    @ptr_inc
    def bxc(self, operand):
        self.B = int(self.B ^ self.C)
    
    @ptr_inc
    def out(self, operand):
        self.outputs.append(self.combo(operand)% 8)
        
    def process(self):
        while self.ptr < len(self.instructions) - 1:
            opcode = self.instructions[self.ptr]
            operand = self.instructions[self.ptr + 1]
            self._opcode(opcode)(operand)

    def __repr__(self):
        return ",".join([str(i) for i in self.outputs])

In [ ]:
def parse(text):
    A = int(re.findall(r"A: (\d+)", text)[0])
    B = int(re.findall(r"B: (\d+)", text)[0])
    C = int(re.findall(r"C: (\d+)", text)[0])
    instr = text.strip().split("Program: ")[1]
    instr = [int(x) for x in instr.split(",")]
    return Computer(A, B, C, instr)

### Part one

In [ ]:
computer = parse(input_text)
computer.process()
print(computer)

### Part two

In [ ]:
def get_i_start(combinations, bit, i_start):
    if bit == len(combinations) - 1:
        return i_start
    i_ = i_start
    for k in range(bit + 1, len(combinations)):
        if(combinations[k]):
            i_+= combinations[k] * 8**k
    return i_

def check_combinations(combination: list, bit, start_computer, i_start):
    current_combination = combination.copy()
    result = []
    for k in range(8):
        computer = copy.deepcopy(start_computer)
        computer.A = get_i_start(combination, bit, i_start) + 8**bit * k
        computer.process()
        if(len(computer.outputs) != len(output)):
            continue
        if computer.outputs[bit] == output[bit]:
            current_combination[bit] = k
            result.append(current_combination.copy())
    return result

start_computer = parse(input_text)
output = start_computer.instructions.copy()
computer = copy.deepcopy(start_computer)
bit = (len(output) - 1)
i_start = 8 **bit
i=i_start
itr = 0
inc = 8 **(len(output) - 1)
combinations = None
combination = [0 for _ in range(bit + 1)]
for bit in range(bit, -1, -1):
    if not combinations:
        combinations = check_combinations(combination, bit, start_computer, i_start)
    else:
        new_combinations = []
        for combination in combinations:
            new_combinations += check_combinations(combination, bit, start_computer, i_start)
        combinations = new_combinations.copy()
A_list = []
for i, _ in enumerate(combinations):
        computer = copy.deepcopy(start_computer)
        starting_A = get_i_start(combinations[i], 0, i_start) + combinations[i][0]
        computer.A = starting_A
        computer.process()
        print(computer)
        if computer.outputs == output:
            print(starting_A)
            A_list.append(starting_A)
min(A_list)